In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, SplineTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style for consistent plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("=" * 80)
print("HOMEWORK: HOUSING MARKET ANALYSIS")
print("Nonparametric Methods: Splines and Decision Trees")
print("=" * 80)

# =============================================================================
# DATA GENERATION (DO NOT MODIFY THIS SECTION)
# =============================================================================

np.random.seed(2024)  # For reproducibility
n = 8000

# Generate housing market data
square_feet = np.random.uniform(800, 4000, n)
lot_size = np.random.uniform(2000, 15000, n) 
age = np.random.uniform(0, 80, n)
bedrooms = np.random.choice([1, 2, 3, 4, 5, 6], n, p=[0.05, 0.15, 0.35, 0.30, 0.10, 0.05])
bathrooms = np.random.choice([1, 1.5, 2, 2.5, 3, 3.5, 4], n, 
                              p=[0.10, 0.10, 0.30, 0.20, 0.15, 0.10, 0.05])
garage_spaces = np.random.choice([0, 1, 2, 3], n, p=[0.10, 0.30, 0.50, 0.10])

# Location features
distance_to_city = np.random.uniform(0.5, 30, n)  # miles
school_rating = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], n,
                                  p=[0.02, 0.03, 0.05, 0.10, 0.15, 0.20, 0.20, 0.15, 0.07, 0.03])
crime_rate = np.random.uniform(1, 100, n)  # per 1000 residents

# Economic variables
unemployment_rate = np.random.uniform(2, 12, n)  # percent
median_income = np.random.uniform(30000, 150000, n)  # dollars

# Generate true house prices with nonlinear relationships
def true_price_function(sqft, age, dist, school, crime, income):
    """
    True data generating process
    - Square footage: linear with diminishing returns at high values
    - Age: depreciation with floor effect
    - Distance: negative effect that diminishes
    - School rating: nonlinear positive effect
    - Crime: negative effect
    - Income: captures neighborhood quality
    """
    base_price = 50000
    
    # Square feet: log relationship (diminishing returns)
    sqft_effect = 80 * np.log(sqft)
    
    # Age: depreciation but levels off for very old homes (vintage effect)
    age_effect = -2000 * np.log(age + 1) + 100 * (age > 50)
    
    # Distance to city: negative effect that diminishes
    dist_effect = -5000 * np.log(dist + 1)
    
    # School rating: nonlinear (quadratic)
    school_effect = 2000 * school + 200 * school**2
    
    # Crime: negative effect
    crime_effect = -200 * crime
    
    # Median income: proxy for neighborhood quality
    income_effect = 1.5 * income
    
    price = (base_price + sqft_effect + age_effect + dist_effect + 
             school_effect + crime_effect + income_effect)
    
    return np.maximum(price, 20000)  # Floor price

price = true_price_function(square_feet, age, distance_to_city, 
                            school_rating, crime_rate, median_income)

# Add noise
price = price * np.random.lognormal(0, 0.15, n)

# Create DataFrame
data = pd.DataFrame({
    'price': price,
    'square_feet': square_feet,
    'lot_size': lot_size,
    'age': age,
    'bedrooms': bedrooms,
    'bathrooms': bathrooms,
    'garage_spaces': garage_spaces,
    'distance_to_city': distance_to_city,
    'school_rating': school_rating,
    'crime_rate': crime_rate,
    'unemployment_rate': unemployment_rate,
    'median_income': median_income
})

# INTRODUCE MISSING VALUES (realistic patterns)
# Missing data is NOT random - this is important!

# Older homes more likely to have missing lot_size (records lost)
missing_lot_mask = (data['age'] > 50) & (np.random.random(n) < 0.15)
data.loc[missing_lot_mask, 'lot_size'] = np.nan

# Crime rate missing for some low-crime areas (not reported)
missing_crime_mask = (data['crime_rate'] < 20) & (np.random.random(n) < 0.10)
data.loc[missing_crime_mask, 'crime_rate'] = np.nan

# School rating missing for some areas
missing_school_mask = np.random.random(n) < 0.08
data.loc[missing_school_mask, 'school_rating'] = np.nan

# Random missing in median_income
missing_income_mask = np.random.random(n) < 0.12
data.loc[missing_income_mask, 'median_income'] = np.nan

print("\n✓ Dataset generated successfully!")
print(f"  Total observations: {len(data)}")
print(f"  Features: {len(data.columns) - 1}")
print(f"  Target: house price")

# =============================================================================
# QUESTION 1: EXPLORATORY DATA ANALYSIS (15 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 1: EXPLORATORY DATA ANALYSIS (15 points)")
print("=" * 80)

# Q1.1 (3 points): Display basic information about the dataset
print("\nQ1.1: Display the first 10 rows and basic dataset info")
print("-" * 60)

### YOUR CODE HERE ###
# Hint: Use .head(), .info(), .describe()




### END YOUR CODE ###

# Q1.2 (4 points): Identify and report missing values
print("\nQ1.2: Identify missing values")
print("-" * 60)
print("Report the number and percentage of missing values for each column.")

### YOUR CODE HERE ###
# Calculate missing values
# Hint: Use .isnull().sum() and create a summary table



### END YOUR CODE ###

# Q1.3 (4 points): Create summary statistics
print("\nQ1.3: Create a summary statistics table")
print("-" * 60)
print("For the target variable (price), report:")
print("  - Mean, Median, Std Dev, Min, Max")
print("  - Skewness (hint: from scipy.stats import skew)")

### YOUR CODE HERE ###




### END YOUR CODE ###

# Q1.4 (4 points): Visualize distributions
print("\nQ1.4: Create visualizations for key variables")
print("-" * 60)
print("Create a 2x3 subplot showing:")
print("  - Histogram of price")
print("  - Histogram of log(price)")
print("  - Scatter: price vs square_feet")
print("  - Scatter: price vs age") 
print("  - Scatter: price vs distance_to_city")
print("  - Box plot: price by school_rating")

### YOUR CODE HERE ###
# Create figure with subplots



### END YOUR CODE ###

# WRITTEN ANSWER Q1.4:
"""
Based on your visualizations, answer the following:
a) Is the price distribution skewed? Should we use log(price)?
b) Which variables appear to have nonlinear relationships with price?
c) Are there any obvious outliers?

YOUR ANSWER:

"""

# =============================================================================
# QUESTION 2: DATA CLEANING AND FEATURE ENGINEERING (20 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 2: DATA CLEANING AND FEATURE ENGINEERING (20 points)")
print("=" * 80)

# Q2.1 (8 points): Handle missing values
print("\nQ2.1: Handle missing values")
print("-" * 60)
print("Impute missing values using appropriate strategies:")
print("  - lot_size: use median")
print("  - crime_rate: use median")
print("  - school_rating: use mode (most common value)")
print("  - median_income: use mean")
print("\nCreate a new DataFrame called 'data_clean' with imputed values.")

### YOUR CODE HERE ###
# Make a copy of the data
data_clean = data.copy()

# Impute missing values
# Hint: Use .fillna() or SimpleImputer from sklearn





### END YOUR CODE ###

# Verify no missing values remain
print("\nMissing values after imputation:")
print(data_clean.isnull().sum())

# Q2.2 (6 points): Create new features
print("\nQ2.2: Feature engineering")
print("-" * 60)
print("Create the following new features:")
print("  1. 'price_per_sqft': price divided by square_feet")
print("  2. 'total_rooms': bedrooms + bathrooms")
print("  3. 'age_category': 'new' (age<10), 'mid' (10-30), 'old' (>30)")
print("  4. 'luxury': 1 if (bedrooms>=4 AND bathrooms>=3), else 0")

### YOUR CODE HERE ###





### END YOUR CODE ###

# Q2.3 (6 points): Create dummy variables
print("\nQ2.3: Create dummy variables for school_rating")
print("-" * 60)
print("Create dummy variables for school_rating (treat as categorical)")
print("Use pd.get_dummies() and add to data_clean")

### YOUR CODE HERE ###
# Hint: pd.get_dummies(data_clean['school_rating'], prefix='school', drop_first=True)




### END YOUR CODE ###

print("\n✓ Data cleaning and feature engineering complete!")
print(f"  Final dataset shape: {data_clean.shape}")

# =============================================================================
# QUESTION 3: TRAIN/TEST SPLIT AND SCALING (10 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 3: TRAIN/TEST SPLIT AND SCALING (10 points)")
print("=" * 80)

# Q3.1 (5 points): Create train/test split
print("\nQ3.1: Split data into training (70%) and test (30%) sets")
print("-" * 60)
print("Use the following features (continuous only for now):")
features_to_use = ['square_feet', 'lot_size', 'age', 'bedrooms', 
                   'bathrooms', 'garage_spaces', 'distance_to_city',
                   'crime_rate', 'unemployment_rate', 'median_income']

### YOUR CODE HERE ###
# Create X (features) and y (target: price)
# Split into train and test sets (test_size=0.3, random_state=42)






### END YOUR CODE ###

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Q3.2 (5 points): Scale the features
print("\nQ3.2: Standardize the features (mean=0, std=1)")
print("-" * 60)
print("Use StandardScaler. Important: fit on training data only!")

### YOUR CODE HERE ###
# Create scaler, fit on training data, transform both train and test




### END YOUR CODE ###

print("\n✓ Data split and scaled!")

# =============================================================================
# QUESTION 4: LINEAR REGRESSION BASELINE (10 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 4: LINEAR REGRESSION BASELINE (10 points)")
print("=" * 80)

# Q4.1 (5 points): Fit linear regression
print("\nQ4.1: Fit a linear regression model")
print("-" * 60)

### YOUR CODE HERE ###
# Fit model, make predictions, calculate MSE and R² for both train and test



### END YOUR CODE ###

print("\nLinear Regression Results:")
print(f"  Training MSE: {train_mse_linear:.2f}")
print(f"  Test MSE: {test_mse_linear:.2f}")
print(f"  Training R²: {train_r2_linear:.4f}")
print(f"  Test R²: {test_r2_linear:.4f}")

# Q4.2 (5 points): Interpret coefficients
print("\nQ4.2: Report and interpret the coefficients")
print("-" * 60)

### YOUR CODE HERE ###
# Create a table showing feature names and coefficients
# Interpret: which features have the largest effect on price?




### END YOUR CODE ###

# WRITTEN ANSWER Q4.2:
"""
Which three features have the largest impact on house price (in absolute value)?
How do you interpret their signs?

YOUR ANSWER:


"""

# =============================================================================
# QUESTION 5: SPLINE REGRESSION (20 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 5: SPLINE REGRESSION (20 points)")
print("=" * 80)

# Q5.1 (8 points): Fit spline for square_feet
print("\nQ5.1: Fit univariate spline regression for square_feet")
print("-" * 60)
print("Use SplineTransformer with n_knots=5, degree=3")

### YOUR CODE HERE ###
# Focus on square_feet only (first column of X_train_scaled)
# Create spline transformer, fit model, evaluate





### END YOUR CODE ###

print(f"\nSpline (square_feet only) Results:")
print(f"  Training MSE: {train_mse_spline_sqft:.2f}")
print(f"  Test MSE: {test_mse_spline_sqft:.2f}")
print(f"  Training R²: {train_r2_spline_sqft:.4f}")
print(f"  Test R²: {test_r2_spline_sqft:.4f}")

# Q5.2 (7 points): Cross-validation for number of knots
print("\nQ5.2: Use cross-validation to select optimal number of knots")
print("-" * 60)
print("Try n_knots from 3 to 12, use 5-fold CV")

### YOUR CODE HERE ###
# For each n_knots value:
#   - Create spline transformer
#   - Transform features
#   - Calculate CV score
# Plot CV score vs n_knots






### END YOUR CODE ###

# Q5.3 (5 points): Multivariate spline model
print("\nQ5.3: Fit spline regression with multiple features")
print("-" * 60)
print("Apply splines to: square_feet, age, distance_to_city")
print("Use optimal n_knots from Q5.2")

### YOUR CODE HERE ###
# Hint: Use ColumnTransformer to apply splines to specific columns
from sklearn.compose import ColumnTransformer

# Define which columns get spline transformation
# square_feet is column 0, age is column 2, distance_to_city is column 6







### END YOUR CODE ###

print(f"\nMultivariate Spline Results:")
print(f"  Training MSE: {train_mse_spline:.2f}")
print(f"  Test MSE: {test_mse_spline:.2f}")
print(f"  Training R²: {train_r2_spline:.4f}")
print(f"  Test R²: {test_r2_spline:.4f}")

# WRITTEN ANSWER Q5.3:
"""
How does the spline model compare to linear regression?
Is the improvement worth the added complexity?

YOUR ANSWER:


"""

# =============================================================================
# QUESTION 6: DECISION TREE (20 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 6: DECISION TREE (20 points)")
print("=" * 80)

# Q6.1 (5 points): Fit a full tree (no pruning)
print("\nQ6.1: Fit a decision tree without constraints")
print("-" * 60)

### YOUR CODE HERE ###
# Fit DecisionTreeRegressor with default parameters
# Evaluate on train and test






### END YOUR CODE ###

print(f"\nFull Tree Results:")
print(f"  Training MSE: {train_mse_tree_full:.2f}")
print(f"  Test MSE: {test_mse_tree_full:.2f}")
print(f"  Training R²: {train_r2_tree_full:.4f}")
print(f"  Test R²: {test_r2_tree_full:.4f}")
print(f"  Number of leaves: {tree_full.get_n_leaves()}")
print(f"  Tree depth: {tree_full.get_depth()}")

# WRITTEN ANSWER Q6.1:
"""
Is the full tree overfitting? How can you tell?

YOUR ANSWER:


"""

# Q6.2 (8 points): Cross-validation for tree depth
print("\nQ6.2: Use cross-validation to find optimal max_depth")
print("-" * 60)
print("Try max_depth from 2 to 20, use 5-fold CV")

### YOUR CODE HERE ###
# For each max_depth:
#   - Fit tree with that max_depth
#   - Calculate CV score
# Plot CV score vs max_depth
# Identify optimal depth









### END YOUR CODE ###

print(f"\nOptimal max_depth: {optimal_depth}")

# Q6.3 (7 points): Fit and evaluate pruned tree
print("\nQ6.3: Fit tree with optimal depth and additional constraints")
print("-" * 60)
print("Use optimal_depth and set min_samples_leaf=20")

### YOUR CODE HERE ###
# Fit pruned tree
# Evaluate on train and test
# Plot the tree structure







### END YOUR CODE ###

print(f"\nPruned Tree Results:")
print(f"  Training MSE: {train_mse_tree:.2f}")
print(f"  Test MSE: {test_mse_tree:.2f}")
print(f"  Training R²: {train_r2_tree:.4f}")
print(f"  Test R²: {test_r2_tree:.4f}")
print(f"  Number of leaves: {tree_pruned.get_n_leaves()}")
print(f"  Tree depth: {tree_pruned.get_depth()}")

# =============================================================================
# QUESTION 7: MODEL COMPARISON AND INTERPRETATION (15 points)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 7: MODEL COMPARISON AND INTERPRETATION (15 points)")
print("=" * 80)

# Q7.1 (5 points): Create comparison table
print("\nQ7.1: Create a comprehensive model comparison table")
print("-" * 60)

### YOUR CODE HERE ###
# Create DataFrame comparing all models
# Include: Model name, Train MSE, Test MSE, Train R², Test R², 
#          Train MAE, Test MAE






### END YOUR CODE ###

print("\nModel Comparison:")
print(comparison_df.to_string(index=False))

# Q7.2 (5 points): Feature importance from tree
print("\nQ7.2: Extract and visualize feature importance from decision tree")
print("-" * 60)

### YOUR CODE HERE ###
# Get feature importances
# Create bar plot
# Report top 5 most important features






### END YOUR CODE ###

# Q7.3 (5 points): Residual analysis
print("\nQ7.3: Create residual plots for all three models")
print("-" * 60)

### YOUR CODE HERE ###
# For each model (linear, spline, tree):
#   - Calculate residuals on test set
#   - Create scatter plot of predicted vs residuals
#   - Create histogram of residuals







### END YOUR CODE ###

# WRITTEN ANSWER Q7.3:
"""
Based on the residual plots:
a) Which model has the most normally distributed residuals?
b) Do you see any patterns in the residuals (heteroskedasticity)?
c) Which model would you recommend for this housing price prediction task? Why?

YOUR ANSWER:
a)

b)

c)


"""

# =============================================================================
# QUESTION 8: ADVANCED ANALYSIS (10 points - BONUS)
# =============================================================================

print("\n" + "=" * 80)
print("QUESTION 8: ADVANCED ANALYSIS (10 bonus points)")
print("=" * 80)

# Q8.1 (5 points): Implement polynomial regression
print("\nQ8.1: Compare splines to polynomial regression")
print("-" * 60)
print("Fit polynomial regression of degrees 2, 3, 4 for square_feet")
print("Compare test MSE to spline regression")

### YOUR CODE HERE ###
from sklearn.preprocessing import PolynomialFeatures

# For each degree, fit model and evaluate







### END YOUR CODE ###

# Q8.2 (5 points): Heterogeneous effects
print("\nQ8.2: Investigate heterogeneous effects")
print("-" * 60)
print("Does the effect of square_feet differ by age category?")
print("Split data into new/mid/old homes and fit separate models")

### YOUR CODE HERE ###
# Use the age_category feature you created earlier
# Fit separate models for each category
# Compare coefficients on square_feet






### END YOUR CODE ###

# WRITTEN ANSWER Q8.2:
"""
How does the square_feet coefficient differ across age categories?
What might explain this pattern economically?

YOUR ANSWER:


"""

# =============================================================================
# FINAL SUBMISSION CHECKLIST
# =============================================================================

print("\n" + "=" * 80)
print("FINAL SUBMISSION CHECKLIST")
print("=" * 80)

print("""
Before submitting, make sure you have:

□ Completed all code sections marked with ### YOUR CODE HERE ###
□ Answered all written questions
□ Created all required visualizations
□ Saved all plots as high-resolution PNG files
□ Created the model comparison table and saved as CSV
□ Written a brief summary (1-2 paragraphs) of your findings
□ Checked that your code runs from start to finish without errors

SUBMISSION REQUIREMENTS:
1. This completed Python file (.py) or Jupyter notebook (.ipynb)
2. PDF report with:
   - All plots
   - All written answers
   - Summary of findings
3. model_comparison.csv file


Good luck!
""")

print("=" * 80)
print("END OF HOMEWORK ASSIGNMENT")
print("=" * 80)

HOMEWORK: HOUSING MARKET ANALYSIS
Nonparametric Methods: Splines and Decision Trees

✓ Dataset generated successfully!
  Total observations: 8000
  Features: 11
  Target: house price

QUESTION 1: EXPLORATORY DATA ANALYSIS (15 points)

Q1.1: Display the first 10 rows and basic dataset info
------------------------------------------------------------

Q1.2: Identify missing values
------------------------------------------------------------
Report the number and percentage of missing values for each column.

Q1.3: Create a summary statistics table
------------------------------------------------------------
For the target variable (price), report:
  - Mean, Median, Std Dev, Min, Max
  - Skewness (hint: from scipy.stats import skew)

Q1.4: Create visualizations for key variables
------------------------------------------------------------
Create a 2x3 subplot showing:
  - Histogram of price
  - Histogram of log(price)
  - Scatter: price vs square_feet
  - Scatter: price vs age
  - Scatter

NameError: name 'X_train' is not defined